In [2]:
import formulas
import storage
import pandas as pd
import os

if __name__ == '__main__':

    website_creator_path = os.getcwd()
    formula_file_path = website_creator_path + os.path.sep + 'formulas.csv'
    order_file_dir = website_creator_path + os.path.sep + 'order_files'
    docs_dir = storage.get_docs_path(website_creator_path)
    df_formulas =  storage.get_formulas(formula_file_path)
    sort_orders = storage.get_sort_orders(order_file_dir)
    
    storage.delete_directory_if_it_exists(docs_dir)

    #Create formula by year summary
    formula_by_year_summary_df = formulas.get_formula_by_year_summary_df(df_formulas)
    storage.create_sub_directories(docs_dir, formula_by_year_summary_df)
    storage.create_index_files(base_dir=docs_dir, folder_regex='.*/Formulas/.*', book_collapse=True, sort_orders=sort_orders)
    # formulas.create_formula_files(docs_dir, df_formulas)

    # df_calculus = formulas.df_calculus_summary(df_formulas)
    # styler_calculus = formulas.styler_calculus_summary(df_calculus, formulas.formulas_on_formula_sheet(df_formulas))
    # display(styler_calculus)

In [19]:
docs_dir

'/home/charl/Onedrive/Documents_Charl/Computer_Technical/Programming_GitHub/AustralianSchoolMaths/content/docs'

In [8]:
_first_index_of_item_in_series('ACdT', sort_orders) is None

True

In [3]:
sort_orders

1               Indices
2            Logarithms
3       Differentiation
4           Integration
5            Statistics
6                 Surds
1               By year
2    Cumulative by year
1              Formulas
1                   ACT
2                   NSW
3                    NT
4                   QLD
5                    SA
6                   VIC
7                    WA
1                Year 9
2           Year 11 Adv
3           Year 12 Adv
4         Year 12 Ext 1
Name: sort value, dtype: object